In [1]:
%load_ext Cython
x_min = -2
x_max = 1
y_min = -1.5
y_max = 1.5
width = 4000
height = 4000
max_iter = 100000

In [2]:
import numpy as np
from random import uniform
import threading
import concurrent
import multiprocessing

class buddhabrot:

    def __init__(self, x_min, x_max, y_min, y_max, width, height, max_iter):
        self.x_min = x_min # real min bound
        self.x_max = x_max # real max bound
        self.y_min = y_min # imaginary min bound
        self.y_max = y_max # imaginary max bound
        self.width = width # pixel width
        self.height = height # pixel height
        self.max_iter = max_iter # maximum iterations per point
        self.file_name = f"buddhabrot_{x_min}_{x_max}_{y_min}_{y_max}_{width}_{height}_{max_iter}.npy"
        try:
            self.grid_array = np.load(self.file_name)
            print('Old file found and loaded...')
        except FileNotFoundError:
            self.grid_array = np.zeros((width,height), np.int64)
            np.save(self.file_name, self.grid_array)
            print('New file created...')
    
    def get_grid_point(self, z_real, z_imag):
        ###
        # compute pixel density
        x_dens = (self.width - 1) / abs(self.x_max - self.x_min)
        y_dens = (self.height - 1) / abs(self.y_max - self.y_min)
        ###
        # compute nearest pixel
        x_pixel = round(x_dens * (z_real - self.x_min))
        y_pixel = self.width - 1 - round(y_dens * (z_imag - self.y_min)) # image starts top left
        ###
        # does nearest pixel exceed image size?
        if x_pixel < 0 or x_pixel > self.width -1 or y_pixel < 0 or y_pixel > self.height - 1:
            return None
        
        return (x_pixel, y_pixel)
    
    def _worker(self, loops):
        ###
        # establish array
        grid_array = np.zeros((self.width, self.height), dtype = np.int64)
        ###
        # iterate over points for this worker
        for _ in range(loops):
            # add point to grid
            self.add_point_to_grid(grid_array)
            
        return grid_array
    
    def add_point_to_grid(self, grid_array):
        ###
        # create random point
        c_real = uniform(self.x_min, self.x_max)
        c_imag = uniform(self.y_min, self.y_max)
        ###
        # first step
        z_real = c_real
        z_imag = c_imag
        ###
        # establish path
        path = {}
        ###
        # start iteration
        for n in range(self.max_iter):
            z_real2 = z_real * z_real
            z_imag2 = z_imag * z_imag
            ###
            # does zn surely diverge?
            if z_real2 + z_imag2 > 4.0:
                for (x,y) in path:
                    grid_array[x,y] += path[(x,y)]
                break
            ###
            # get nearest pixel
            grid_point = self.get_grid_point(z_real, z_imag)
            ###
            # pixel already visited?
            if grid_point in path:
                    path[grid_point] += 1
            elif grid_point is not None:
                path[grid_point] = 1
            ###
            # prepare new step
            z_imag = 2 * z_real*z_imag + c_imag
            z_real = z_real2 - z_imag2 + c_real

    def get_array(self, update=True):
        ###
        # update from file
        if update:
            self.grid_array = np.load(self.file_name)
        return self.grid_array
    
    def load_file(self):
        self.grid_array = np.load(self.file_name)
        print('From file updated...')
        
    def save_file(self):
        np.save(self.file_name, self.grid_array)
        print('File saved...')
            
    def sharpen_buddha(self, loops=1, num_workers=1):
        ###
        # start multiprocessing
        with concurrent.futures.ProcessPoolExecutor() as executor:
            ###
            # create workers
            workers = [executor.submit(buddhabrot._worker, self, loops) for _ in range(num_workers)]
            ###
            # combine results
            for f in concurrent.futures.as_completed(workers):
                self.grid_array += f.result()
                print(f"{loops} new points evaluated...")
                self.save_file()

In [ ]:
%%time
buddha = buddhabrot(x_min, x_max, y_min, y_max, width, height, max_iter)
buddha.sharpen_buddha(loops=1000, num_workers=100)
array = buddha.get_array()

Old file found and loaded...


In [ ]:
%%cython
from PIL import Image

cpdef create_image(array):
    (width, height) = array.shape
    img = Image.new('RGB', (width, height))
    max_val = array.max()
    pixels = []
    for y in range(height):
        for x in range(width):
            gray = int(255 * (1 - array[x,y]/max_val))
            pixels.append((gray, gray, gray))
    img.putdata(pixels)
    return img

In [ ]:
img = create_image(array)

In [ ]:
img.show()

In [ ]:
img.save('buddhabrot_' + str(width) + '_' + str(height) + '_' + str(max_iter) + '.png')

In [ ]:
buddha.get_array()